In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from scipy import signal
from scipy.interpolate import interp1d
from cerebralcortex import Kernel
from scipy.stats import skew,kurtosis,mode
CC = Kernel("/home/jupyter/cc3_conf/", study_name='moral')

/cerebralcortex/kessel_jupyter_virtualenv/cc33/lib64/python3.6/site-packages/cerebralcortex/core/data_manager/raw/data.py:67: DeprecationWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  self.fs = pa.hdfs.connect(self.hdfs_ip, self.hdfs_port)


In [21]:
CC.list_streams()

['accelerometer--org.md2k.motionsense--motion_sense--left_wrist',
 'accelerometer--org.md2k.motionsense--motion_sense--right_wrist',
 'accelerometer--org.md2k.motionsense--motion_sense--right_wrist.activity.all.three',
 'annotation--org.md2k.motionsense--motion_sense--moral_brushing_flossing',
 'gyroscope--org.md2k.motionsense--motion_sense--left_wrist',
 'gyroscope--org.md2k.motionsense--motion_sense--right_wrist']

In [22]:
import tempfile
import tensorflow 

def interpolate_acl(a,window_size=20,fs_now=25,fs_new=20):
    x_now = np.linspace(0,window_size,a.shape[0])
    f = interp1d(x_now,a,axis=0,fill_value='extrapolate')
    x_new = np.linspace(0,window_size,window_size*fs_new)
    return f(x_new)

class ModelWrapperPickable:

    def __init__(self, model):
        self.model = model

    def __getstate__(self):
        model_str = ''
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            tensorflow.keras.models.save_model(self.model, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            self.model = tensorflow.keras.models.load_model(fd.name)

            

def compute_magnitude_and_activity(data,
                                   Fs = 16,
                                   window_size = 20,
                                   stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all',
                                   new_Fs = 20,
                                   acceptable_percentage = .8,
                                   filename1 = './models/activity_estimator_both_study_magnitude_final_5_labels.hdf5',
                                   filename2 = './models/activity_estimator_wisdm_magnitude_final_5_labels.hdf5',
                                   filename3 = './models/activity_estimator_dalia_magnitude_final_5_labels.hdf5'):
            
    model = load_model(filename1)

    model_wrapper= ModelWrapperPickable(model)


    model_wisdm = load_model(filename2)

    model_wrapper_wisdm= ModelWrapperPickable(model_wisdm)
    
    model_dalia = load_model(filename3)

    model_wrapper_dalia = ModelWrapperPickable(model_dalia)


    data = data.select('localtime','timestamp','x','y','z','user','version')

    data = data.withColumn('magnitude',F.sqrt(F.pow(F.col('x'),2)+F.pow(F.col('y'),2)+F.pow(F.col('z'),2))).drop('x','y','z')

    data = data.withColumn('time',F.col('timestamp').cast('double'))

    data = data.withColumn('magnitude_time',F.array('time','magnitude')).drop('time','magnitude')

    groupbycols = ['user','version',
                   F.window('timestamp',windowDuration=str(window_size)+' seconds', startTime='0 seconds',slideDuration=str(window_size//2)+' seconds')]
    data_windowed_10 = data.groupBy(groupbycols).agg(F.collect_list('magnitude_time'),
                                                     F.collect_list('localtime'))

    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(magnitude_time)','magnitude_time')
    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(localtime)','localtime')
    
#     data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(red)','red')
#     data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(infrared)','infrared')
#     data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(green)','green')

#     qfunction = F.udf(compute_quality,StringType())
#     data_windowed_10 = data_windowed_10.withColumn('quality',qfunction(data_windowed_10['red'],data_windowed_10['infrared'],data_windowed_10['green']))

#     data_windowed_10 = data_windowed_10.drop('red','infrared','green')
    
#     data_windowed_10 = data_windowed_10.filter(F.col('quality')=='Worn').drop('quality')

    
    data_windowed_10 = data_windowed_10.withColumn('localtime',F.col('localtime').getItem(0))

    data_windowed_10 = data_windowed_10.withColumn('length',F.size('magnitude_time'))

    data_windowed_10 = data_windowed_10.filter(F.col('length')>=acceptable_percentage*window_size*Fs).drop('length')

    data_windowed_10 = data_windowed_10.withColumn('timestamp',F.col('window').start).withColumn('start',F.col('window').start).withColumn('end',F.col('window').end).drop('window')

    data_windowed_10  = data_windowed_10.withColumn('day',F.date_format('localtime',"yyyyMMdd"))
    
#     data_windowed_10.show(1,False)
#     data_windowed_10.printSchema()
    final_activity_list = ['Stationery','Stairs','Exercise','Walking','Sports']
    final_activity_dict = {a:i for i,a in enumerate(final_activity_list)}
    final_activity_dict_reverse = {i:a for i,a in enumerate(final_activity_list)}
    activity_list_wisdm = ['Stationery','Stairs','Exercise','Walking','Sports']
    activity_list_dalia = ['Stationery','Stairs','Exercise','Walking','Sports']
#     activity_list_wisdm = ['Sitting','Stairs','Jogging','Typing','Standing','Walking','Brushing','Eating','Drinking','Kicking',
#                     'Playing Catch','Dribbling','Writing','Clapping','Folding Clothes']
#     activity_list_dalia = ['Driving','Sitting','Stairs','Walking']
#     print(data_windowed_10.schema)
    schema = StructType(list(data_windowed_10.schema)[:2]+list(data_windowed_10.schema)[3:]+[StructField("magnitude", ArrayType(DoubleType())),
                                                                                            StructField("prediction", StringType()),
                                                                                            StructField("prediction_wisdm", StringType()),
                                                                                            StructField("prediction_dalia", StringType()),
                                                                                            StructField("mean", DoubleType()),
                                                                                            StructField("std", DoubleType()),
                                                                                            StructField("skew", DoubleType()),
                                                                                            StructField("kurtosis", DoubleType())])
#     print(schema)

    
#     def smooth_predictions(df):
#         if df.shape[0]<3:
#             return pd.DataFrame([],columns=columns)
#         for name in ['prediction','prediction_wisdm','prediction_dalia']:
#             y_activities = list(df[name])
#             m_act = mode(y_activities)[0][0]
#             df[name] = [m_act]*df.shape[0]
#         return df

    columns = [a.name for a in schema.fields]
    print(columns)
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def interpolate_acl_magnitude(df):
        df['magnitude_time'] = df['magnitude_time'].apply(lambda b:np.array([np.array(a) for a in b]).reshape(len(b),2))
        df['magnitude_time'] = df['magnitude_time'].apply(lambda a:a[a[:,0].argsort()])
        df['magnitude_time'] = df['magnitude_time'].apply(lambda a:a[:,1].reshape(a.shape[0],1))
        
        df['magnitude'] = df['magnitude_time'].apply(lambda a:interpolate_acl(a).reshape(1,window_size*new_Fs,1))
        X = np.concatenate(list(df['magnitude']))
        y_pred = model_wrapper.model.predict(X).argmax(axis=1)
        df['prediction'] = list(y_pred.reshape(-1))
        df['prediction'] = df['prediction'].apply(lambda a:final_activity_dict_reverse[a])

        y_pred = model_wrapper_wisdm.model.predict(X).argmax(axis=1)
        df['prediction_wisdm'] = list(y_pred.reshape(-1))
        df['prediction_wisdm'] = df['prediction_wisdm'].apply(lambda a:activity_list_wisdm[a])
        
        y_pred = model_wrapper_dalia.model.predict(X).argmax(axis=1)
        df['prediction_dalia'] = list(y_pred.reshape(-1))
        df['prediction_dalia'] = df['prediction_dalia'].apply(lambda a:activity_list_dalia[a])
        
        df['magnitude'] = df['magnitude'].apply(lambda a:a.reshape(-1)) 
        df['mean'] = df['magnitude'].apply(lambda a:np.mean(a))
        df['std'] = df['magnitude'].apply(lambda a:np.std(a))
        df['skew'] = df['magnitude'].apply(lambda a:skew(a))
        df['kurtosis'] = df['magnitude'].apply(lambda a:kurtosis(a))
        df = df[columns]
#         df = df.groupby(pd.Grouper(key='timestamp',freq=str(3*window_size)+'S'),as_index=False).apply(smooth_predictions)
        return df

    data_interpolated = data_windowed_10.groupBy(['user','version','day']).apply(interpolate_acl_magnitude)
    schema = data_interpolated.schema
#     data_interpolated.show(1,False)
    stream_metadata = Metadata()
    stream_metadata.set_name(stream_name+'.activity.all.three').set_description("Activity Computed")
    for field in schema.fields:
        stream_metadata.add_dataDescriptor(
            DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
        )
    stream_metadata.add_module(
        ModuleMetadata().set_name("activity datastream") \
        .set_attribute("url", "https://md2k.org").set_author(
            "Md Azim Ullah", "mullah@memphis.edu"))
    stream_metadata.is_valid()
    data_interpolated.printSchema()
    ds = DataStream(data=data_interpolated,metadata=stream_metadata)
    return ds
# CC.save_stream(ds,overwrite=True)

# stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all'
stream_name = 'accelerometer--org.md2k.motionsense--motion_sense--right_wrist'
data = CC.get_stream(stream_name)
data.show(5,False)
data_activity = compute_magnitude_and_activity(data,stream_name=stream_name)
CC.save_stream(data_activity,overwrite=True)

+-----------------------+-----------+----------+-----------+-----------------------+----+-------+
|timestamp              |x          |y         |z          |localtime              |user|version|
+-----------------------+-----------+----------+-----------+-----------------------+----+-------+
|2017-03-16 19:55:51.584|-0.23120117|-0.9250488|-0.13598633|2017-03-16 13:55:51.584|820c|1      |
|2017-03-16 19:55:51.644|-0.23266602|-0.9182129|-0.14160156|2017-03-16 13:55:51.644|820c|1      |
|2017-03-16 19:55:51.704|-0.23046875|-0.9152832|-0.1352539 |2017-03-16 13:55:51.704|820c|1      |
|2017-03-16 19:55:51.764|-0.2290039 |-0.9172363|-0.14428711|2017-03-16 13:55:51.764|820c|1      |
|2017-03-16 19:55:51.824|-0.2241211 |-0.9111328|-0.13647461|2017-03-16 13:55:51.824|820c|1      |
+-----------------------+-----------+----------+-----------+-----------------------+----+-------+
only showing top 5 rows

['user', 'version', 'localtime', 'timestamp', 'start', 'end', 'day', 'magnitude', 'prediction

/cerebralcortex/kessel_jupyter_virtualenv/cc33/lib64/python3.6/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


root
 |-- user: string (nullable = true)
 |-- version: integer (nullable = false)
 |-- localtime: timestamp (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- start: timestamp (nullable = true)
 |-- end: timestamp (nullable = true)
 |-- day: string (nullable = true)
 |-- magnitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- prediction: string (nullable = true)
 |-- prediction_wisdm: string (nullable = true)
 |-- prediction_dalia: string (nullable = true)
 |-- mean: double (nullable = true)
 |-- std: double (nullable = true)
 |-- skew: double (nullable = true)
 |-- kurtosis: double (nullable = true)



True

In [23]:
data = CC.get_stream(stream_name+'.activity.all.three')

In [24]:
data.groupBy('prediction').count().show(5,False)

+----------+-------+
|prediction|count  |
+----------+-------+
|Stairs    |9494   |
|Walking   |26275  |
|Stationery|1190228|
|Sports    |89905  |
|Exercise  |633    |
+----------+-------+



Resampling mperf data to 20Hz and computes the magnitude and applies activity recognition model

In [ ]:
import tempfile
import tensorflow 

def interpolate_acl(a,window_size=20,fs_now=25,fs_new=20):
    x_now = np.linspace(0,window_size,a.shape[0])
    f = interp1d(x_now,a,axis=0,fill_value='extrapolate')
    x_new = np.linspace(0,window_size,window_size*fs_new)
    return f(x_new)

class ModelWrapperPickable:

    def __init__(self, model):
        self.model = model

    def __getstate__(self):
        model_str = ''
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            tensorflow.keras.models.save_model(self.model, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            self.model = tensorflow.keras.models.load_model(fd.name)


def isDatapointsWithinRange(red,infrared,green,window_size=20):
        red = np.asarray(red, dtype=np.float32)
        infrared = np.asarray(infrared, dtype=np.float32)
        green = np.asarray(green, dtype=np.float32)
        a =  len(np.where((red >= 20000)& (red<=200000))[0]) < .33*window_size*25
        b = len(np.where((infrared >= 80000)& (infrared<=230000))[0]) < .33*window_size*25
        c = len(np.where((green >= 500)& (green<=25000))[0]) < .33*window_size*25
        if a and b and c:
            return False
        return True            

            
def compute_quality(red,infrared,green):
        """
        :param window: a window containing list of DataPoints
        :return: an integer reptresenting the status of the window 0= attached, 1 = not attached
        """
        if len(red)==0:
            return 'Not Worn' #not attached
        if not isDatapointsWithinRange(red,infrared,green):
            return 'Not Worn'
        red_mean = np.mean(red)
        ir_mean = np.mean(infrared)
        green_mean = np.mean(green)
        if red_mean < 5000 and ir_mean < 5000:
            return 'Not Worn'
        if not (red_mean>green_mean and ir_mean>red_mean):
            return 'Not Worn'
        diff = 30000
        if red_mean<130000:
            diff = 10000
        if red_mean - green_mean <= diff:
            return 'Not Worn'
        if ir_mean - red_mean <= diff:
            return 'Not Worn'
        return 'Worn'            
            

def compute_magnitude_and_activity(data,
                                   Fs = 25,
                                   window_size = 20,
                                   stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all',
                                   new_Fs = 20,
                                   acceptable_percentage = .8,
                                   filename1 = './models/activity_estimator_both_study_magnitude_final_5_labels.hdf5',
                                   filename2 = './models/activity_estimator_wisdm_magnitude_final_5_labels.hdf5',
                                   filename3 = './models/activity_estimator_dalia_magnitude_final_5_labels.hdf5'):
            
    model = load_model(filename1)

    model_wrapper= ModelWrapperPickable(model)


    model_wisdm = load_model(filename2)

    model_wrapper_wisdm= ModelWrapperPickable(model_wisdm)
    
    model_dalia = load_model(filename3)

    model_wrapper_dalia = ModelWrapperPickable(model_dalia)


    data = data.select('localtime','timestamp','aclx','acly','aclz','user','version','red','infrared','green')

    data = data.withColumn('magnitude',F.sqrt(F.pow(F.col('aclx'),2)+F.pow(F.col('acly'),2)+F.pow(F.col('aclz'),2))).drop('aclx','acly','aclz')

    data = data.withColumn('time',F.col('timestamp').cast('double'))

    data = data.withColumn('magnitude_time',F.array('time','magnitude')).drop('time','magnitude')

    groupbycols = ['user','version',
                   F.window('timestamp',windowDuration=str(window_size)+' seconds', startTime='0 seconds')]
    data_windowed_10 = data.groupBy(groupbycols).agg(F.collect_list('magnitude_time'),
                                                     F.collect_list('localtime'),
                                                    F.collect_list('red'),
                                                    F.collect_list('infrared'),
                                                    F.collect_list('green'))

    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(magnitude_time)','magnitude_time')
    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(localtime)','localtime')
    
    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(red)','red')
    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(infrared)','infrared')
    data_windowed_10 = data_windowed_10.withColumnRenamed('collect_list(green)','green')

    qfunction = F.udf(compute_quality,StringType())
    data_windowed_10 = data_windowed_10.withColumn('quality',qfunction(data_windowed_10['red'],data_windowed_10['infrared'],data_windowed_10['green']))

    data_windowed_10 = data_windowed_10.drop('red','infrared','green')
    
    data_windowed_10 = data_windowed_10.filter(F.col('quality')=='Worn').drop('quality')

    
    data_windowed_10 = data_windowed_10.withColumn('localtime',F.col('localtime').getItem(0))

    data_windowed_10 = data_windowed_10.withColumn('length',F.size('magnitude_time'))

    data_windowed_10 = data_windowed_10.filter(F.col('length')>=acceptable_percentage*window_size*Fs).drop('length')

    data_windowed_10 = data_windowed_10.withColumn('timestamp',F.col('window').start).withColumn('start',F.col('window').start).withColumn('end',F.col('window').end).drop('window')

    data_windowed_10  = data_windowed_10.withColumn('day',F.date_format('localtime',"yyyyMMdd"))

    final_activity_list = ['Stationery','Stairs','Exercise','Walking','Sports']
    final_activity_dict = {a:i for i,a in enumerate(final_activity_list)}
    final_activity_dict_reverse = {i:a for i,a in enumerate(final_activity_list)}
    activity_list_wisdm = ['Stationery','Stairs','Exercise','Walking','Sports']
    activity_list_dalia = ['Stationery','Stairs','Exercise','Walking','Sports']
#     activity_list_wisdm = ['Sitting','Stairs','Jogging','Typing','Standing','Walking','Brushing','Eating','Drinking','Kicking',
#                     'Playing Catch','Dribbling','Writing','Clapping','Folding Clothes']
#     activity_list_dalia = ['Driving','Sitting','Stairs','Walking']
#     print(data_windowed_10.schema)
    schema = StructType(list(data_windowed_10.schema)[:2]+list(data_windowed_10.schema)[3:]+[StructField("magnitude", ArrayType(DoubleType())),
                                                                                            StructField("prediction", StringType()),
                                                                                            StructField("prediction_wisdm", StringType()),
                                                                                            StructField("prediction_dalia", StringType()),
                                                                                            StructField("mean", DoubleType()),
                                                                                            StructField("std", DoubleType()),
                                                                                            StructField("skew", DoubleType()),
                                                                                            StructField("kurtosis", DoubleType())])
#     print(schema)

    
    def smooth_predictions(df):
        if df.shape[0]<3:
            return pd.DataFrame([],columns=columns)
        for name in ['prediction','prediction_wisdm','prediction_dalia']:
            y_activities = list(df[name])
            m_act = mode(y_activities)[0][0]
            df[name] = [m_act]*df.shape[0]
        return df

    columns = [a.name for a in schema.fields]
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def interpolate_acl_magnitude(df):
        df['magnitude_time'] = df['magnitude_time'].apply(lambda b:np.array([np.array(a) for a in b]).reshape(len(b),2))
        df['magnitude_time'] = df['magnitude_time'].apply(lambda a:a[a[:,0].argsort()])
        df['magnitude_time'] = df['magnitude_time'].apply(lambda a:a[:,1].reshape(a.shape[0],1))
        
        df['magnitude'] = df['magnitude_time'].apply(lambda a:interpolate_acl(a).reshape(1,window_size*new_Fs,1))
        X = np.concatenate(list(df['magnitude']))
        y_pred = model_wrapper.model.predict(X).argmax(axis=1)
        df['prediction'] = list(y_pred.reshape(-1))
        df['prediction'] = df['prediction'].apply(lambda a:final_activity_dict_reverse[a])

        y_pred = model_wrapper_wisdm.model.predict(X).argmax(axis=1)
        df['prediction_wisdm'] = list(y_pred.reshape(-1))
        df['prediction_wisdm'] = df['prediction_wisdm'].apply(lambda a:activity_list_wisdm[a])
        
        y_pred = model_wrapper_dalia.model.predict(X).argmax(axis=1)
        df['prediction_dalia'] = list(y_pred.reshape(-1))
        df['prediction_dalia'] = df['prediction_dalia'].apply(lambda a:activity_list_dalia[a])
        
        df['magnitude'] = df['magnitude'].apply(lambda a:a.reshape(-1)) 
        df['mean'] = df['magnitude'].apply(lambda a:np.mean(a))
        df['std'] = df['magnitude'].apply(lambda a:np.std(a))
        df['skew'] = df['magnitude'].apply(lambda a:skew(a))
        df['kurtosis'] = df['magnitude'].apply(lambda a:kurtosis(a))
        df = df[columns]
#         df = df.groupby(pd.Grouper(key='timestamp',freq=str(3*window_size)+'S'),as_index=False).apply(smooth_predictions)
        return df

    data_interpolated = data_windowed_10.groupBy(['user','version','day']).apply(interpolate_acl_magnitude)
    schema = data_interpolated.schema
    stream_metadata = Metadata()
    stream_metadata.set_name(stream_name+'.activity.all.three').set_description("Activity Computed")
    for field in schema.fields:
        stream_metadata.add_dataDescriptor(
            DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
        )
    stream_metadata.add_module(
        ModuleMetadata().set_name("activity datastream") \
        .set_attribute("url", "https://md2k.org").set_author(
            "Md Azim Ullah", "mullah@memphis.edu"))
    stream_metadata.is_valid()
    data_interpolated.printSchema()
    ds = DataStream(data=data_interpolated,metadata=stream_metadata)
    return ds
# CC.save_stream(ds,overwrite=True)

stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all'
data = CC.get_stream(stream_name)
data.show(5,False)
data_activity = compute_magnitude_and_activity(data,stream_name=stream_name)
CC.save_stream(data_activity,overwrite=True)

In [ ]:
data = CC.get_stream('org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all.three')

In [ ]:
data.select('prediction').groupBy('prediction').count().show(5,False)

In [ ]:
data  = CC.get_stream('org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all')

In [ ]:
data.groupBy('prediction_dalia').count().show(5,False)

In [ ]:
right_data = CC.get_stream('org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity.all')

count_data = right_data.select('user','prediction','day').groupBy(['user','prediction','day']).count()
df = count_data.toPandas()
import pickle
pickle.dump(df,open('./data/right_wrist_count.p','wb'))

In [ ]:
df_user_mean = df.groupby(['prediction','user'],as_index=False).mean()

In [ ]:
df_user_mean.groupby('prediction').mean()

In [ ]:
right_data = CC.get_stream('org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity')

count_data = right_data.select('user','prediction_wisdm','day').groupBy(['user','prediction_wisdm','day']).count()
df = count_data.toPandas()
import pickle
pickle.dump(df,open('./data/right_wrist_count_wisdm.p','wb'))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pickle.load(open('./data/right_wrist_count.p','rb'))

In [ ]:
directory = './data/20/walking/testing/'

In [ ]:
import os
import pickle

In [ ]:
count = 0
for f in os.listdir(directory):
    data = pickle.load(open(directory+f,'rb'))
    if data.shape[0]//3>100:
        count+=1
print(count)

In [ ]:
df['count'] = df['count'].apply(lambda a:(a*10)/3600)
df['prediction'][df['prediction'].isin(['Sitting','Eating'])] = ['Sitting']*df[df['prediction'].isin(['Sitting','Eating'])].shape[0]

In [ ]:
df_walking = df[df['prediction']=='Sitting']

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(df_walking[['user','count']].groupby('user').quantile(.5)['count'],20)
plt.title('Maximum walking hours for a single day')
plt.xlabel('Hours')
plt.tight_layout()
plt.savefig('./images/Maximumwalk.png')
plt.show()

In [ ]:
df_walking.groupby('user').max()

In [ ]:
df_user_mean = df.groupby(['user','prediction'],as_index=False).mean()
plt.rcParams.update({'font.size':20})
plt.figure(figsize=(20,10))
sns.boxplot(x='prediction',y='count',data=df_user_mean)
plt.xticks(rotation='60')
plt.ylabel('Hours')
plt.savefig('./images/user_mean_per_day.png')
plt.show()


In [ ]:
plt.figure(figsize=(20,10))
plt.hist(df_user_mean['count'][df_user_mean['prediction'].isin(['Walking','Stairs','Sports','Exercise'])],300,density=True)
plt.xlim([0,2])
plt.title('Moving Hours per day distribution')
plt.xlabel('Hours')
plt.savefig('./images/moving_hours_per_day.png')
plt.show()

In [ ]:
# final_activity_list = ['Brushing','Cycling','Sports','Eating','Driving',
#                        'Exercise','Sitting','Stairs','Standing','Walking']
def percentage(df):
    activities = np.unique(df['prediction'])
    result  = []
    for a in activities:
        result.append(100*df['count'][df['prediction']==a].values[0]/df['count'].sum())
    users = [df.user.values[0]]*len(result)
    return pd.DataFrame({'activities':activities,'user':users,'percentage':result})

df_user_percentage = df_user_mean.groupby('user',as_index=False).apply(percentage)

In [ ]:
plt.rcParams.update({'font.size':20})
plt.figure(figsize=(20,10))
sns.boxplot(x='activities',y='percentage',data=df_user_percentage)
plt.xticks(rotation=60)
plt.savefig('./images/user_percentage_per_day.png')
plt.show()

In [ ]:
stream_name = 'org.md2k.feature.motionsensehrv.decoded.leftwrist.all'

In [ ]:
data = CC.get_stream(stream_name)

In [ ]:
data.select('user').distinct().count()

In [ ]:
data_activity.count()

In [ ]:
61692/(25*3600)

In [ ]:
data.count()

In [ ]:
data_activity.show(4,False)

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf
import numpy as np
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
np.random.seed(100)

In [ ]:
import tensorflow as tf

In [ ]:
import os

In [ ]:
os.listdir('hdfs://dantooine10dot.memphis.edu/user/mullah')

In [ ]:
import shutil

In [ ]:
shutil.rmtree('./data/right_wrist/')